In [1]:
"""purpose - create a persistent DuckDB database and store data from a couple datasets (i.e., as if they had been imported from CSV flatfiles) in it, each as their own table"""
from contextlib import contextmanager
import json
import pathlib as pl
import sys
import warnings

import appdirs
import duckdb
import pandas as pd
from pydataset import data
from IPython.display import Markdown


# HACK the path for allow this notebook to import server's modules
THIS_DIR = globals()['_dh'][0]
REPO_ROOT = THIS_DIR.parent
SRC_DIR = (REPO_ROOT/'src').resolve()
#print(f"{THIS_DIR=}\n{REPO_ROOT=}\n{SRC_DIR=}")
sys.path.append(str(SRC_DIR))

from backend.sql import base
from backend.sql.duck import DuckDBMonitorMiddleware
from backend.sql.postgres import PostgresMonitorMiddleware
from pages.utils.etc import load_config
config = load_config()


@contextmanager
def ignore_warnings():
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        yield

In [2]:
base.init(wipe=True)
display(PostgresMonitorMiddleware.ask_available_tables())

['pg_foobar', 'mfgwh_mx908_exit_tests']

In [3]:
df = PostgresMonitorMiddleware.get_dataframe('pg_foobar')
if df is not None:
    # df = df.pivot_table(
    #     index=['report_id', 'created_at'],
    #     columns=['attribute'],
    #     values=['value'],
    #     aggfunc='first'
    # )
    display(df)


,id,name
0,1,this is a name


In [4]:
with ignore_warnings(), duckdb.connect(base.DUCKDB.PATH) as conn:
    usage = pd.read_sql("SELECT * FROM administrative.table_catalog;", conn)
display(usage)

,table_name,table_description,table_type,owned_by,created,updates,updated,hits,last_hit,is_pseudo_table,external_type,etc
0,iris,"A 1936 dataset prepared by statistician Ronald Fisher, contains multivariate data desc...",system,unknown,2025-08-27 07:14:26.984,0,2025-08-27 07:14:26.984,0,2025-08-27 07:14:26.984,False,None,{}
1,economics,This dataset was produced from US economic time series data available from the Federal...,system,unknown,2025-08-27 07:14:26.989,0,2025-08-27 07:14:26.989,0,2025-08-27 07:14:26.989,False,None,{}
2,octoprint,A sensor stream from my personal Octoprint 3D printer server via HomeAssistant...,system,jbechen,2025-08-27 07:14:26.999,0,2025-08-27 07:14:26.999,0,2025-08-27 07:14:26.999,False,None,{}
3,pg_foobar,None,system,unknown,2025-08-27 07:14:27.047,0,2025-08-27 07:14:27.047,1,2025-08-27 07:14:27.114,True,postgres,"{""cluster"":""localhost"",""database"":""foobar"",""sql"":""SELECT * FROM my_table;""}"
4,mfgwh_mx908_exit_tests,None,system,unknown,2025-08-27 07:14:27.053,0,2025-08-27 07:14:27.053,0,2025-08-27 07:14:27.053,True,postgres,"{""cluster"":""localhost"",""database"":""mfgwh"",""sql"":""SELECT\n rep.id \""report_id\"",\n ..."
